In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from time import sleep
from tqdm import tqdm
import re

In [2]:
def getSoup(link):###vytvoříme funkci, která získá z URL adresy text
    sleep(0.05)
    r = requests.get(link)
    return BeautifulSoup(r.text,'lxml')

In [3]:
def getAllLinks(link):###najdeme tabulku linků
    soup = getSoup(link)
    table = soup.find('table', {'id':'reportable'})
    tds = table.findAll('td', {'align':'left'})
    a = [td.find('a') for td in tds]
    res = [i for i in a if i]
    return res

In [4]:
links = getAllLinks('http://tennisabstract.com/reports/atpRankings.html')###získáme všechny linky

In [5]:
b = [str(links[i])for i in range(len(links))]
c = [i.replace('"', '!') for i in b]
d = [o.split("!") for o in c]
flinks = [d[p][1] for p in range(len(d))]

In [6]:
Namesss = [flink.replace('http://www.tennisabstract.com/cgi-bin/player.cgi?p=', '') for flink in flinks]###získáme jména
Names=Namesss[:150]

In [7]:
finallist = []###udeláme list, ve kterém jsou listy z údaji jednotlivých hráčů
ranx = 0
for nam in Names:
    ranx += 1
    soop = getSoup('http://www.minorleaguesplits.com/tennisabstract/cgi-bin/frags/'+nam+'.js')
    playerlist = []
    name_str = nam
    name_list = []
    name_list = re.findall('[A-Z][^A-Z]*', name_str)
    namstr = ' '.join(map(str, name_list))
    playerlist.append(namstr)
    playerlist.append(ranx)
    if (soop.find('table', {'id':'tour-years'})):
        statstable = soop.find('table', {'id':'tour-years'})
        statsdfs = pd.read_html(str(statstable), index_col = 0)
        statsdf = statsdfs[0]
        playerlist.append(statsdf.at['Career', 'Hld%'])
        playerlist.append(statsdf.at['Career', 'Brk%'])
    else:
        playerlist.append("No data")
        playerlist.append("No data")
    finallist.append(playerlist)
    
print(finallist)

[['Novak Djokovic', 1, '85.8%', '32.1%'], ['Rafael Nadal', 2, '85.9%', '33.4%'], ['Dominic Thiem', 3, '83.2%', '23.8%'], ['Daniil Medvedev', 4, '82.4%', '25.5%'], ['Roger Federer', 5, '88.8%', '26.7%'], ['Stefanos Tsitsipas', 6, '85.6%', '19.8%'], ['Alexander Zverev', 7, '81.5%', '24.7%'], ['Andrey Rublev', 8, '80.2%', '25.5%'], ['Diego Sebastian Schwartzman', 9, '72.3%', '32.1%'], ['Matteo Berrettini', 10, '86.8%', '17.4%'], ['Gael Monfils', 11, '80.8%', '25.1%'], ['Denis Shapovalov', 12, '83.7%', '18.6%'], ['Roberto Bautista Agut', 13, '80.3%', '26.5%'], ['Milos Raonic', 14, '91.3%', '15.9%'], ['David Goffin', 15, '77.8%', '27.6%'], ['Pablo Carreno Busta', 16, '77.8%', '24.6%'], ['Fabio Fognini', 17, '72.0%', '27.2%'], ['Stanislas Wawrinka', 18, '82.8%', '22.7%'], ['Felix Auger Aliassime', 19, '81.1%', '20.0%'], ['Karen Khachanov', 20, '82.6%', '21.0%'], ['Grigor Dimitrov', 21, '82.6%', '21.8%'], ['Christian Garin', 22, '75.7%', '23.9%'], ['Alex De Minaur', 23, '79.6%', '23.5%'], ['J

In [8]:
finalldf = pd.DataFrame(finallist, columns = ['Name', 'Rank', 'Hld%', 'Brk%'])###uděláme dataframe
finalldf

,Name,Rank,Hld%,Brk%
0,Novak Djokovic,1,85.8%,32.1%
1,Rafael Nadal,2,85.9%,33.4%
2,Dominic Thiem,3,83.2%,23.8%
3,Daniil Medvedev,4,82.4%,25.5%
4,Roger Federer,5,88.8%,26.7%
...,...,...,...,...
145,Francisco Cerundolo,146,61.5%,26.9%
146,Oscar Otte,147,80.6%,20.0%
147,Marc Andrea Huesler,148,82.0%,19.6%
148,Ivo Karlovic,149,92.0%,8.5%
